**IN PROGRESS**

# Baseline: Accuracy of Patroller Reverts (Probable Vandalism) - Recent Changes

[TASK: T348863](https://phabricator.wikimedia.org/T348863)

In [3]:
def get_rc_min_timestamp(wiki):

    query = """
        SELECT
            MIN(rc_timestamp) AS ts
        FROM
            recentchanges
    """
    
    result = wmf.mariadb.run(query, wiki)
    return int(result.ts.values[0])

In [ ]:
%%time

warnings.filterwarnings('ignore')

unpatrolled_rc = pd.DataFrame()

for wiki in ['enwiki', 'idwiki']:
    unpatrolled_rc_query = f"""
    SELECT
        rc_id,
        rc_timestamp AS rc_ts,
        rc_title,
        rc_namespace,
        rc_new,
        rc_this_oldid,
        rc_type,
        rc_deleted,
        actor_name AS rc_user_name
    FROM
        recentchanges rc
    JOIN
        actor a ON rc.rc_actor = a.actor_id 
    WHERE
        rc_bot = 0
        AND rc_patrolled = 0
        AND actor_name NOT LIKE '%bot%'
    """

unpatrolled_rc = wmf.mariadb.run(unpatrolled_rc_query, dbs='enwiki')
unpatrolled_rc_query.info()

In [16]:
def get_recent_changes(wiki):
    
    rc_start = get_rc_min_timestamp(wiki)
    
    rc_query = f"""
    WITH
        recent_changes AS (
            SELECT
                rc_id,
                rc_timestamp AS rc_ts,
                rc_title,
                rc_namespace,
                rc_new,
                rc_this_oldid,
                rc_type,
                rc_deleted,
                rc_patrolled,
                actor_name AS rc_user_name
            FROM
                recentchanges rc
            JOIN
                actor a ON rc.rc_actor = a.actor_id 
            WHERE
                rc_bot = 0
                AND rc_patrolled IN (0, 1)
                AND actor_name NOT LIKE '%bot%'
        ),

        patrol_logs AS (
            SELECT
                log_id,
                log_timestamp,
                log_title,
    CAST(
        SUBSTRING(
            log_params, 
            LOCATE('"4::curid";s:', log_params) + CHAR_LENGTH('"4::curid";s:') + LOCATE(':', SUBSTRING(log_params, LOCATE('"4::curid";s:', log_params) + CHAR_LENGTH('"4::curid";s:'))) + 1,
            LOCATE('"', SUBSTRING(log_params FROM LOCATE('"4::curid";s:', log_params) + CHAR_LENGTH('"4::curid";s:') + LOCATE(':', SUBSTRING(log_params, LOCATE('"4::curid";s:', log_params) + CHAR_LENGTH('"4::curid";s:'))) + 1)) - 1
        ) AS UNSIGNED) AS log_rev_id,
                log_page,
                u.user_name AS patroller_user_name,
            CASE
                WHEN user_editcount < 100 THEN '0-99'
                WHEN user_editcount BETWEEN 100 AND 999 THEN '100-999'
                WHEN user_editcount BETWEEN 1000 AND 4999 THEN '1000-4999'
                WHEN user_editcount >= 5000 THEN '5000+'
            END AS reviewer_edit_bucket,
            CASE
                WHEN is_bot THEN TRUE
                ELSE FALSE
            END AS is_patroller_bot
            FROM
                logging log
            JOIN
                actor a
                ON log.log_actor = a.actor_id
            JOIN
                user u
                ON a.actor_user = u.user_id
            LEFT JOIN
                (
                    SELECT
                        ug_user AS user_id,
                        TRUE AS is_bot
                    FROM
                        user_groups
                    WHERE
                        ug_group = 'bot'
                ) bots
                ON bots.user_id = u.user_id
            WHERE
                log_type = 'patrol'
                AND log_timestamp >= {rc_start}
        ),

        rc_patrols AS (
            SELECT
                *
            FROM
                recent_changes rc
            LEFT JOIN
                patrol_logs pl
                ON rc.rc_this_oldid = pl.log_rev_id)

    SELECT 
        * 
    FROM
        rc_patrols
    """

    return wmf.mariadb.run(rc_query, dbs=wiki)

In [17]:
%%time

warnings.filterwarnings('ignore')

idwiki_rc = get_recent_changes('idwiki')

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


CPU times: user 33.6 s, sys: 51.6 s, total: 1min 25s
Wall time: 4h 59s
